<a href="https://colab.research.google.com/github/Rakshit-Shetty/Resnet-Implementation/blob/master/RESNET_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The Resnet Research paper can be accessed from here https://arxiv.org/pdf/1512.03385v1.pdf

In [0]:
import torch
import torchvision